In [3]:
import os
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import librosa
from tqdm import tqdm
from sklearn.metrics import roc_curve


In [4]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

SAMPLE_RATE = 16000
SEGMENT_LEN = 32000
FRAME_SIZE = 160

BATCH_SIZE = 1
LR = 1e-4
EPOCHS = 10

DEVICE

Using device: cuda


'cuda'

In [5]:
def circular_shift(arr, shift):
    return np.roll(arr, shift)

def least_number(a, b):
    g = math.gcd(a, b)
    return a // g, b // g
def generate_matrix(N, M, P, Q):
    row = np.concatenate([np.ones(M), np.zeros(N - M)])
    rows = []
    for _ in range(M):
        rows.append(row)
        row = circular_shift(row, M)
    return np.array(rows)
def apply_cs(audio, sensing_matrix, frame_size):
    n_frames = len(audio) // frame_size
    out = []
    for i in range(n_frames):
        frame = audio[i*frame_size:(i+1)*frame_size]
        out.append(sensing_matrix @ frame)
    return np.concatenate(out)
def build_sensing_matrix(N, compression_ratio):
    M = int(compression_ratio * N)
    P, Q = least_number(M, N)
    return generate_matrix(N, M, P, Q)


In [10]:
import gc
SAMPLE_RATE = 16000
FRAME_SIZE = 160
SEGMENT_LEN = 64000   # or whatever you use

AUDIO_DIR = "LA\\ASVspoof2019_LA_eval\\flac"
SAVE_ROOT = "TestData"

compression_levels = {
    # "cs_25": 0.25,
    # "cs_50": 0.50,
    "cs_75": 0.75
}


os.makedirs(SAVE_ROOT, exist_ok=True)

for folder, ratio in compression_levels.items():
    save_dir = os.path.join(SAVE_ROOT, folder)
    os.makedirs(save_dir, exist_ok=True)

    sensing_matrix = build_sensing_matrix(FRAME_SIZE, ratio)

    print(f"\nProcessing {folder} ({int(ratio*100)}%)")

    for fname in tqdm(os.listdir(AUDIO_DIR)):
        if not fname.endswith(".flac"):
            continue

        utt = os.path.splitext(fname)[0]
        audio_path = os.path.join(AUDIO_DIR, fname)

        try:
            # safer loading
            audio, _ = librosa.load(
                audio_path,
                sr=SAMPLE_RATE,
                mono=True
            )

            if audio.size == 0:
                raise ValueError("Empty audio")

        except Exception as e:
            print(f"[SKIPPED] {fname}: {e}")
            continue

        # Pad / trim
        if len(audio) < SEGMENT_LEN:
            audio = np.pad(audio, (0, SEGMENT_LEN - len(audio)))
        else:
            audio = audio[:SEGMENT_LEN]

        # Apply CS
        compressed_audio = apply_cs(audio, sensing_matrix, FRAME_SIZE)

        # Save
        save_path = os.path.join(save_dir, utt + ".npy")
        np.save(save_path, compressed_audio)

        # Prevent file-handle & memory buildup
        del audio, compressed_audio
        gc.collect()


Processing cs_75 (75%)


 20%|█▉        | 14333/71933 [23:30<1:34:26, 10.16it/s]


OSError: 48000 requested and 0 written

In [1]:
import os
import torch

ROOT_DIR = "Models"   # change if needed

sub_dirs = ["25% CS", "50% CS", "75% CS"]

for sub in sub_dirs:
    sub_path = os.path.join(ROOT_DIR, sub)

    for file in os.listdir(sub_path):
        if file.endswith(".pth"):
            pth_path = os.path.join(sub_path, file)

            # Load the model/checkpoint
            checkpoint = torch.load(pth_path, map_location="cpu")

            # Create .pt filename
            pt_name = file.replace(".pth", ".pt")
            pt_path = os.path.join(sub_path, pt_name)

            # Save as .pt
            torch.save(checkpoint, pt_path)

            print(f"Converted: {pth_path} → {pt_path}")



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\bhatt\miniconda3\envs\torch118\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\bhatt\miniconda3\envs\torch118\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\bhatt\miniconda3\envs\torch118\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\bhatt\miniconda3\envs\torch118\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
 

Converted: Models\25% CS\aasist_cs_25.pth → Models\25% CS\aasist_cs_25.pt
Converted: Models\25% CS\rawformerL_cs_25.pth → Models\25% CS\rawformerL_cs_25.pt
Converted: Models\25% CS\rawnet2_cs_25.pth → Models\25% CS\rawnet2_cs_25.pt
Converted: Models\25% CS\rawtfnet_16_cs_25.pth → Models\25% CS\rawtfnet_16_cs_25.pt
Converted: Models\25% CS\rawtfnet_32_cs_25.pth → Models\25% CS\rawtfnet_32_cs_25.pt
Converted: Models\50% CS\aasist_cs_50.pth → Models\50% CS\aasist_cs_50.pt
Converted: Models\50% CS\rawformerL_50_cs.pth → Models\50% CS\rawformerL_50_cs.pt
Converted: Models\50% CS\rawnet2_cs_50.pth → Models\50% CS\rawnet2_cs_50.pt
Converted: Models\50% CS\rawtfnet_cs_best32.pth → Models\50% CS\rawtfnet_cs_best32.pt
Converted: Models\50% CS\rawtfnet_cs_best_16.pth → Models\50% CS\rawtfnet_cs_best_16.pt
Converted: Models\75% CS\aasist_cs_75.pth → Models\75% CS\aasist_cs_75.pt
Converted: Models\75% CS\rawformerL_cs_75.pth → Models\75% CS\rawformerL_cs_75.pt
Converted: Models\75% CS\rawnet2_cs_75